# Creating your own Backflow transformation

We present here how to create your own backflow transformation and use it in QMCTorch.
During the import you must import the base class of the backflow kernel. We aso create a H2 molecule

In [ ]:
import torch
from qmctorch.scf import Molecule
from qmctorch.wavefunction import SlaterJastrow
from qmctorch.wavefunction.orbitals.backflow.kernels import BackFlowKernelBase
from qmctorch.wavefunction.orbitals.backflow import BackFlowTransformation

mol = Molecule(atom='H 0. 0. 0; H 0. 0. 1.', unit='bohr', redo_scf=True)

We can then use this base class to create a new backflow transformation kernel.
This is done in the same way one would create a new neural network layer in pytorch

In [ ]:
from torch import nn 
class MyBackflowKernel(BackFlowKernelBase):
    def __init__(self, mol, cuda, size=16):
        super().__init__(mol, cuda)
        self.fc1 = nn.Linear(1, size, bias=False)
        self.fc2 = nn.Linear(size, 1, bias=False)
    def forward(self, x):
        original_shape = x.shape
        x = x.reshape(-1,1)
        x = self.fc2(self.fc1(x))
        return x.reshape(*original_shape)

This backflow transformation consists of two fully connected layers. The calculation of the first and second derivative are then done via automatic differentiation as implemented in the `BackFlowKernelBase` class. To use this new kernel in the `SlaterJastrow` wave function ansatz we first need to instantiate a backflow layer using this kernel

In [ ]:
backflow = BackFlowTransformation(mol, MyBackflowKernel, backflow_kernel_kwargs={'size': 8})

We can then use this backflow transformation in the call of the wave function:

In [ ]:
wf = SlaterJastrow(mol, backflow=backflow)

In [ ]:
pos = torch.rand(10, wf.nelec*3)
print(wf(pos))